# Set pos for a 2R robot

In the previous practice we dealt with passive structures. Let's dive into our workout routine with active systems. For the sake of simplicity, within this script we deal with step signal as a reference. 

As always we start with uploading needed libraries

In [1]:
from os import path
import mujoco
import mujoco_viewer
import numpy as np

Let us declare constant variables such as simulation time `SIMEND`, step numbers `STEP_NUM`, name of  xml file `MODEL_NAME`, and MuJoCo entities: `model` and `data`

In [2]:
SIMEND = 10
STEP_NUM = 1000
MODEL_NAME = 'open_chain_robot.xml'

ROOT = path.abspath('')
FOLDER_PATH = path.join(ROOT, '')
MODEL_PATH = path.join(FOLDER_PATH, MODEL_NAME)

model = mujoco.MjModel.from_xml_path(MODEL_PATH)
data = mujoco.MjData(model)

Our first activity will involve implementing position-velocity control for the servo, followed by advancing to torque control. 

To understand the next piece of code you need to open the xml file and to take a look at `actuator` container. You'll see there a few actuators: (0) `motor`, (1) `position`, (2) velocity for the joint A, and (3) `motor`, (4) `position`, (5) `velocity` for the joint B. To implement servo control we need only position and velocity types, i.e., actuators with IDs 1 and 2 for the joint A, and with IDs 4 and 5 for the joint B. Ids 0 and 3 are needed for torque control.

In [3]:
def set_position_servo(mjmodel, actuator_no, kp):
    mjmodel.actuator_gainprm[actuator_no, 0] = kp
    mjmodel.actuator_biasprm[actuator_no, 1] = -kp


def set_velocity_servo(mjmodel, actuator_no, kv):
    mjmodel.actuator_gainprm[actuator_no, 0] = kv
    mjmodel.actuator_biasprm[actuator_no, 2] = -kv


def controller(mjmodel, mjdata, q_A: float, q_B: float):

    ID_POS_A = 1
    ID_VEL_A = 2
    ID_POS_B = 4
    ID_VEL_B = 5

    set_position_servo(mjmodel, ID_POS_A, 100)
    set_velocity_servo(mjmodel, ID_VEL_A, 10)
    mjdata.ctrl[ID_POS_A] = q_A

    set_position_servo(mjmodel, ID_POS_B, 40)
    set_velocity_servo(mjmodel, ID_VEL_B, 10)
    mjdata.ctrl[ID_POS_B] = q_B

At this stage we have to create a viewer entity, set control callback, and write a piece of code for the `mj_step` command

In [4]:
viewer = mujoco_viewer.MujocoViewer(model, data,
                                    title="2R_robot",
                                    width=1920,
                                    height=1080)

for i in range(STEP_NUM):
    if viewer.is_alive:
        time_prev = data.time
        while (data.time - time_prev < 1.0/60.0):
            mujoco.set_mjcb_control(controller(model, data, np.pi/2, np.pi/2))
            mujoco.mj_step(model, data)            

        if (data.time >= SIMEND):
            break
        viewer.render()
    else:
        break

viewer.close()   

And now let us take a look at an example how we can implement the same task, but with torque controlled actuators 

In [5]:
def set_torque_servo(actuator_no, flag):
    if (flag == 0):
        model.actuator_gainprm[actuator_no, 0] = 0
    else:
        model.actuator_gainprm[actuator_no, 0] = 1


def controller(model, mjdata, q_A: float, q_B: float):
    ID_JOINT_A = 0
    ID_JOINT_B = 1
    TAU_ID_A = 0
    TAU_ID_B = 3
    KP = 100
    KD = 10

    set_torque_servo(0, 1)
    mjdata.ctrl[TAU_ID_A] = KP*(q_A-mjdata.qpos[ID_JOINT_A]) + KD*(0-mjdata.qvel[ID_JOINT_A])
    mjdata.ctrl[TAU_ID_B] = KP*(q_B-mjdata.qpos[ID_JOINT_B]) + KD*(0-mjdata.qvel[ID_JOINT_B])

Create a viewer entity, set control callback, and write a piece of code for the `mj_step` command

In [6]:
viewer = mujoco_viewer.MujocoViewer(model, data,
                                    title="2R_robot",
                                    width=1920,
                                    height=1080)

for i in range(STEP_NUM):
    if viewer.is_alive:
        time_prev = data.time
        while (data.time - time_prev < 1.0/60.0):
            mujoco.set_mjcb_control(controller(model, data, np.pi/2, np.pi/2))
            mujoco.mj_step(model, data)
            
        if (data.time >= SIMEND):
            break
        viewer.render()
    else:
        break

viewer.close()    

Try to change `kp` and `kv`. Also you can change the position reference as extra activity. 